# Получение информации о фильмах и критиках с Кинопоиска

In [52]:
import math
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np
import PySimpleGUI as sg


# Пустые листы для добавления данных с последующим добавлением в общий data frame
all_reviews_name = []
all_reviews_link = []
all_reviews_date = []
all_reviews_film = []
all_opinions = []
all_press_names = []
all_films_names = []
all_films_years = []
all_films_countries = []
all_films_platforms = []
all_films_genries = []
all_films_creators = []
all_films_main_directors = []
all_films_prices = []
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    # XPATHs
    show_selector_xpath = '//select[@class="navigator_per_page"]'
    max_show_selector_xpath = '//option[@value="200"]'
    num_reviews_xpath = '//div[@class="pagesFromTo"]'
    press_name_xpath = '//h1[@class="breadcrumbs__head"]'
    positives_xpath = '//div[contains(@class, "item positive")]'
    negatives_xpath = '//div[contains(@class, "item negative")]'
    film_name_xpath = '//span[@data-tid="75209b22" or @data-tid="2da92aed"]'
    film_year_xpath = '//a[contains(@href, "movies/year")]'
    film_countries_xpath = '//a[contains(@href, "movies/country")]'
    film_platforms_xpath = '//a[contains(@href, "movies/company-originals")]'
    film_genre_xpath = '//a[contains(@href, "movies/genre")]'
    film_creators_xpath = '//a[contains(@href, "/name/") and @data-tid="603f73a4"]'
    film_price_xpath = '//span[contains(@class, "film-rating-value styles_root")]'
    # Проходим все СМИ
    for press_id in range(1, 156 + 1):
        driver.get(f'https://www.kinopoisk.ru/press/source/{press_id}')
        try:
            press_name =  driver.find_elements(By.XPATH, press_name_xpath)[1].text
        except IndexError:  # обработка удаленных URL СМИ, например https://www.kinopoisk.ru/press/source/5
            continue
        try:
            driver.find_element(By.XPATH, show_selector_xpath).click()
            driver.find_element(By.XPATH, max_show_selector_xpath).click()
            num_reviews = driver.find_element(By.XPATH, num_reviews_xpath).text.split()[2]
            num_pages = math.ceil(int(num_reviews) / 200)
        except NoSuchElementException:
            num_pages = 1
        # Проходим все страницы для выбранного СМИ
        for page_number in range(1, num_pages + 1):
            driver.get(f'https://www.kinopoisk.ru/press/source/{press_id}/page/{page_number}/#list')
            # Проходим все рецензии критиков на странице
            for press_opinion_xpath in [positives_xpath, negatives_xpath]:
                reviews_name = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="name"]/a')
                all_reviews_name.extend([review.text for review in reviews_name])
                reviews_link = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="fullText"]/a')
                all_reviews_link.extend([review.get_attribute('href') for review in reviews_link])
                reviews_date = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="date"]')
                all_reviews_date.extend([review.text for review in reviews_date])
                reviews_film = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="subject"]/a')
                all_reviews_film.extend([review.get_attribute('href') for review in reviews_film])
                press_opinions = [re.search(r'positive|negative', press_opinion_xpath)[0] for _ in range(len(reviews_name))]
                all_press_names.extend(press_name for _ in range(len(reviews_name)))
                all_opinions.extend(press_opinions)
    # Проходим все фильмы, на которые ссылались рецензии критиков            
    for link in all_reviews_film:
        # прогресс бар
        sg.one_line_progress_meter('Progress meter', i+1, len(all_reviews_film) + 1, '-key-')
        film_id = re.search(r'\d+', link)[0]
        driver.get(f'https://www.kinopoisk.ru/film/{film_id}/')
        try:
            film_name = driver.find_element(By.XPATH, film_name_xpath).text
        except NoSuchElementException:
            driver.get(f'https://www.kinopoisk.ru/series/{film_id}/')
            film_name = driver.find_element(By.XPATH, film_name_xpath).text
        film_year = driver.find_element(By.XPATH, film_year_xpath).text
        film_countries = [country.text for country in driver.find_elements(By.XPATH, film_countries_xpath)]
        try:
            film_platforms = [platform.text for platform in driver.find_elements(By.XPATH, film_platforms_xpath)]
        except NoSuchElementException:
            film_platforms= None
        film_genries = [genre.text for genre in driver.find_elements(By.XPATH, film_genre_xpath)]
        film_creators = {creator.text for creator in driver.find_elements(By.XPATH, film_creators_xpath)}
        film_main_director = driver.find_element(By.XPATH, film_creators_xpath).text
        film_price = driver.find_element(By.XPATH, film_price_xpath).text
        all_films_names.append(film_name)
        all_films_years.append(film_year)
        all_films_countries.append(film_countries)
        all_films_platforms.append(film_platforms)
        all_films_genries.append(film_genries)
        all_films_creators.append(film_creators)
        all_films_main_directors.append(film_main_director)
        all_films_prices.append(film_price)

KeyboardInterrupt: 

# Создание pd.DataFrame с полученной информацией

In [77]:
data = {'press_name': all_press_names, 
        'positive/negative': all_opinions, 
        'film_name': all_films_names, 
        'film_year': all_films_years,
        'film_director': all_films_main_directors,
        'film_price': all_films_prices,
        'film_countries': all_films_countries,
        'film_platforms': all_films_platforms,
        'film_genries': all_films_genres,
        'film_creators': all_films_creators,
        'links': all_reviews_film,
        'reviewers': all_reviews_name, 
        'full_link': all_reviews_link}
df = pd.DataFrame(data)

,press_name,positive/negative,film_name,film_year,film_director,film_price,film_countries,film_platforms,film_genries,film_creators,links,reviewers,full_link
11 августа 2022,Коммерсантъ,positive,История кино: Новое поколение (2021),2021,Марк Казинс,7.2,[Великобритания],[],[документальный],"{Марк Казинс, Тимо Ленджер, Клара Глинн, Джек ...",https://www.kinopoisk.ru/film/4929765/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5503459
18 мая 2022,Коммерсантъ,positive,Нулевой пациент,2022,Сергей Трофимов,8.4,[Россия],[Кинопоиск],"[драма, детектив]","{Игорь Вдовин, Андрей Геласимов, Александра Ре...",https://www.kinopoisk.ru/film/4642708/press/,Михаил Трофименков,https://www.kommersant.ru/doc/5355793
17 мая 2022,Коммерсантъ,positive,Нелегал (2021),2021,Дмитрий Давыдов,6.1,[Россия],[],[драма],"{Павел Разумов, Дмитрий Давыдов, Дмитрий Никол...",https://www.kinopoisk.ru/film/4929712/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5355032
5 мая 2022,Коммерсантъ,positive,Где Анна Франк (2021),2021,Ари Фольман,7.0,"[Бельгия, Люксембург, Франция, Нидерланды, Изр...",[],"[мультфильм, драма, семейный, история]","{Тристан Оливер, Анна Франк, Джани Тилтгес, Yv...",https://www.kinopoisk.ru/film/819722/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5339842
2 марта 2022,Коммерсантъ,positive,Герой (2021),2021,Асгар Фархади,6.8,"[Иран, Франция]",[],[драма],"{Араш Рамезани, Негар Немати, Реми Бура, Али Г...",https://www.kinopoisk.ru/film/4439600/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5238415
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12 декабря 2021,Cinemarticle,negative,И просто так,2021,Майкл Патрик Кинг,6.6,[США],[HBO Max],"[драма, мелодрама, комедия]","{Крис Коллинз, Шери Байландер, Джиллиан Робесп...",https://www.kinopoisk.ru/film/4357563/press/,Екатерина Высоцких,https://t.me/cinemarticle/7822
24 июня 2022,Тинькофф Журнал,positive,Род мужской (2022),2022,Алекс Гарленд,6.1,[Великобритания],[],"[ужасы, драма]","{Джейк Робертс, Джофф Барроу, Кахэл Бэннон, Эн...",https://www.kinopoisk.ru/film/4397114/press/,Ефим Гугнин,https://journal.tinkoff.ru/men-moment/
23 мая 2022,Тинькофф Журнал,positive,Нулевой пациент,2022,Сергей Трофимов,8.4,[Россия],[Кинопоиск],"[драма, детектив]","{Игорь Вдовин, Андрей Геласимов, Александра Ре...",https://www.kinopoisk.ru/film/4642708/press/,Роман Персианинов,https://journal.tinkoff.ru/patient-zero/
6 апреля 2022,Тинькофф Журнал,positive,Лунный рыцарь,2022,Мохамед Диаб,7.2,[США],[Disney+],"[фэнтези, боевик, приключения]","{Аарон Мурхед, Грант Кертис, Мохамед Диаб, Пит...",https://www.kinopoisk.ru/film/1320559/press/,Ефим Гугнин,https://journal.tinkoff.ru/moon-knight-who/


# Обработка и анализ данных

In [9]:
import math
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np
import PySimpleGUI as sg

df = pd.read_csv('/home/pyvchk/PycharmProjects/kinopoisk/kinopoisk_press_reserch/kinopoisk_reserch.csv', index_col=0)
df.head()

,press_name,positive/negative,film_name,film_year,film_director,film_price,film_countries,film_platforms,film_genries,film_creators,links,reviewers,full_link
11 августа 2022,Коммерсантъ,positive,История кино: Новое поколение (2021),2021,Марк Казинс,7.2,['Великобритания'],[],['документальный'],"{'Марк Казинс', 'Тимо Ленджер', 'Клара Глинн',...",https://www.kinopoisk.ru/film/4929765/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5503459
18 мая 2022,Коммерсантъ,positive,Нулевой пациент,2022,Сергей Трофимов,8.4,['Россия'],['Кинопоиск'],"['драма', 'детектив']","{'Игорь Вдовин', 'Андрей Геласимов', 'Александ...",https://www.kinopoisk.ru/film/4642708/press/,Михаил Трофименков,https://www.kommersant.ru/doc/5355793
17 мая 2022,Коммерсантъ,positive,Нелегал (2021),2021,Дмитрий Давыдов,6.1,['Россия'],[],['драма'],"{'Павел Разумов', 'Дмитрий Давыдов', 'Дмитрий ...",https://www.kinopoisk.ru/film/4929712/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5355032
5 мая 2022,Коммерсантъ,positive,Где Анна Франк (2021),2021,Ари Фольман,7.0,"['Бельгия', 'Люксембург', 'Франция', 'Нидерлан...",[],"['мультфильм', 'драма', 'семейный', 'история']","{'Тристан Оливер', 'Анна Франк', 'Джани Тилтге...",https://www.kinopoisk.ru/film/819722/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5339842
2 марта 2022,Коммерсантъ,positive,Герой (2021),2021,Асгар Фархади,6.8,"['Иран', 'Франция']",[],['драма'],"{'Араш Рамезани', 'Негар Немати', 'Реми Бура',...",https://www.kinopoisk.ru/film/4439600/press/,Юлия Шагельман,https://www.kommersant.ru/doc/5238415
